In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
mask_annotations = json.load(open('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/head_tail_masks.json'))

In [ ]:
left = {'head': (np.mean([point['x'] for point in mask_annotations[0]['Label']['Head'][0]['geometry']]), \
                      np.mean([point['y'] for point in mask_annotations[0]['Label']['Head'][0]['geometry']])), \
             'tail': (np.mean([point['x'] for point in mask_annotations[0]['Label']['Tail'][0]['geometry']]), \
                      np.mean([point['y'] for point in mask_annotations[0]['Label']['Tail'][0]['geometry']]))}

In [ ]:
right = {'head': (np.mean([point['x'] for point in mask_annotations[1]['Label']['Head'][0]['geometry']]), \
                      np.mean([point['y'] for point in mask_annotations[1]['Label']['Head'][0]['geometry']])), \
             'tail': (np.mean([point['x'] for point in mask_annotations[1]['Label']['Tail'][0]['geometry']]), \
                      np.mean([point['y'] for point in mask_annotations[1]['Label']['Tail'][0]['geometry']]))}

In [ ]:
left_image_path = '/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010008_rectified/right_sotra-small-pen_0_1538489067394.jpg'
right_image_path = '/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010008_rectified/left_sotra-small-pen_0_1538489067394.jpg'

In [ ]:
plt.imshow(Image.open(left_image_path))
plt.scatter(left['head'][0], left['head'][1])
plt.scatter(left['tail'][0], left['tail'][1])
plt.show()

In [ ]:
plt.imshow(Image.open(right_image_path))
plt.scatter(right['head'][0], right['head'][1])
plt.scatter(right['tail'][0], right['tail'][1])
plt.show()

In [ ]:
Image.open(right_image_path).size

# convert to depth

In [ ]:
from scipy.interpolate import interp2d

In [ ]:
def convert_to_world_point(x, y, d):
    image_center_x = 3000 / 2.0 #depth_map.shape[1] / 2.0
    image_center_y = 4096 / 2.0# depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (image_sensor_width / 3000)
    sensor_z = px_z * (image_sensor_height / 4096)
    
    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return (world_x, world_y, world_z)

In [ ]:
focal_length = 0.0107
baseline = 0.135
pixel_size_m = 3.45 * 1e-6 
focal_length_pixel = focal_length / pixel_size_m
image_sensor_width = 0.01412
image_sensor_height = 0.01412

In [ ]:
disparities = [left['head'][0] - right['head'][0], left['tail'][0] - right['tail'][0]]
print(disparities)

In [ ]:
depth = focal_length_pixel*baseline / np.array(disparities)
print(depth)

In [ ]:
world_head = convert_to_world_point(left['head'][1], left['head'][0], depth[0])
print(world_head)

In [ ]:
world_tail = convert_to_world_point(left['tail'][1], left['tail'][0], depth[1])
print(world_tail)

In [ ]:
dist = np.linalg.norm(np.array(world_head) - np.array(world_tail))
print(dist)